In [ ]:
%%capture
!pip install transformers

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
import transformers

import pandas as pd
import numpy as np

from torch.utils.data import Dataset
from tqdm import trange
import random
from tabulate import tabulate

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/data2.csv')

df = df.reset_index(drop=True)

FileNotFoundError: ignored

In [ ]:
tokenizer = BertTokenizer.from_pretrained("DeepPavlov/rubert-base-cased", do_lower_case = False)

In [ ]:
text = df['text']
labels = df['text_type']

NameError: ignored

In [ ]:
token_id = []
attention_masks = []

def preprocessing(input_text, tokenizer):
    '''
    Returns <class transformers.tokenization_utils_base.BatchEncoding> with the following fields:
      - input_ids: list of token ids
      - token_type_ids: list of token type ids
      - attention_mask: list of indices (0,1) specifying which tokens should considered by the model (return_attention_mask = True).
    '''
    return tokenizer.encode_plus(
                          input_text,
                          add_special_tokens = True,
                          max_length = 32,
                          pad_to_max_length = True,
                          return_attention_mask = True,
                          return_tensors = 'pt',
                          truncation=True
                    )

In [ ]:
for sample in text:
    encoding_dict = preprocessing(sample, tokenizer)
    token_id.append(encoding_dict['input_ids'])
    attention_masks.append(encoding_dict['attention_mask'])


token_id = torch.cat(token_id, dim = 0)
attention_masks = torch.cat(attention_masks, dim = 0)
labels = torch.tensor(labels)

In [ ]:
def print_rand_sentence_encoding():
  '''Displays tokens, token IDs and attention mask of a random text sample'''
  index = random.randint(0, len(text) - 1)
  tokens = tokenizer.tokenize(tokenizer.decode(token_id[index]))
  token_ids = [i.numpy() for i in token_id[index]]
  attention = [i.numpy() for i in attention_masks[index]]

  table = np.array([tokens, token_ids, attention]).T
  print(tabulate(table,
                 headers = ["Tokens", "Token IDs", "Attention Mask"],
                 tablefmt = "fancy_grid"))

print_rand_sentence_encoding()

╒═════════════╤═════════════╤══════════════════╕
│ Tokens      │   Token IDs │   Attention Mask │
╞═════════════╪═════════════╪══════════════════╡
│ [CLS]       │         101 │                1 │
├─────────────┼─────────────┼──────────────────┤
│ Ност        │       88874 │                1 │
├─────────────┼─────────────┼──────────────────┤
│ ##альг      │       46672 │                1 │
├─────────────┼─────────────┼──────────────────┤
│ ##ия        │        1577 │                1 │
├─────────────┼─────────────┼──────────────────┤
│ по          │        1516 │                1 │
├─────────────┼─────────────┼──────────────────┤
│ СССР        │        5576 │                1 │
├─────────────┼─────────────┼──────────────────┤
│ вызывает    │       21248 │                1 │
├─────────────┼─────────────┼──────────────────┤
│ у           │         875 │                1 │
├─────────────┼─────────────┼──────────────────┤
│ гражданских │       23674 │                1 │
├─────────────┼─────

In [ ]:
val_ratio = 0.2

# Indices of the train and validation splits stratified by labels
train_idx, val_idx = train_test_split(
    np.arange(len(labels)),
    test_size = val_ratio,
    )

# Train and validation sets
train_set = TensorDataset(token_id[train_idx],
                          attention_masks[train_idx],
                          labels[train_idx])

val_set = TensorDataset(token_id[val_idx],
                        attention_masks[val_idx],
                        labels[val_idx])

In [ ]:
train_idx

array([33502, 29184,  2759, ..., 23700, 24254,  3758])

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
def bert_experiment(batch_size, lr, epochs):
    # Recommended batch size: 16, 32.
    # Recommended learning rates (Adam): 5e-5, 3e-5, 2e-5.
    # Recommended number of epochs: 2, 3, 4.
    # See: https://arxiv.org/pdf/1810.04805.pdf

    train_dataloader = DataLoader(
            train_set,
            sampler = RandomSampler(train_set),
            batch_size = batch_size,
        )

    validation_dataloader = DataLoader(
                val_set,
                sampler = SequentialSampler(val_set),
                batch_size = batch_size
            )
    model = BertForSequenceClassification.from_pretrained(
    "DeepPavlov/rubert-base-cased",
    num_labels = 2,
    output_attentions = False,
    output_hidden_states = False
    )

    optimizer = torch.optim.AdamW(model.parameters(), lr = lr, eps = 1e-08)

    # Run on GPU
    model.cuda()

    for _ in trange(epochs, desc = "Epoch"):

        val_labels = []
        val_preds = []

        # ========== Training ==========

        # Set model to training mode
        model.train()

        # Tracking variables
        tr_loss = 0
        nb_tr_examples, nb_tr_steps = 0, 0

        for step, batch in enumerate(train_dataloader):
            batch = tuple(t.to(device) for t in batch)
            b_input_ids, b_input_mask, b_labels = batch
            optimizer.zero_grad()
            # Forward pass
            train_output = model(b_input_ids,
                                token_type_ids = None,
                                attention_mask = b_input_mask,
                                labels = b_labels)
            # Backward pass
            train_output.loss.backward()
            optimizer.step()
            # Update tracking variables
            tr_loss += train_output.loss.item()
            nb_tr_examples += b_input_ids.size(0)
            nb_tr_steps += 1

        # ========== Validation ==========

        # Set model to evaluation mode
        model.eval()

        for batch in validation_dataloader:

            batch = tuple(t.to(device) for t in batch)
            b_input_ids, b_input_mask, b_labels = batch
            with torch.no_grad():
              # Forward pass
              eval_output = model(b_input_ids,
                                  token_type_ids = None,
                                  attention_mask = b_input_mask)
            label_ids = b_labels.to('cpu').numpy().flatten()
            logits = np.argmax(eval_output.logits.detach().cpu().numpy(), axis = 1).flatten()
            val_preds = np.concatenate([val_preds, logits])
            val_labels = np.concatenate([val_labels, label_ids])

        conf_matrix = confusion_matrix(val_labels, val_preds)

        val_accuracy = accuracy_score(val_labels, val_preds)
        print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
        print('\n\t - Validation accuracy (epoch): {:.4f}'.format(val_accuracy))
        print(classification_report(val_labels, val_preds))

    return mismatched_indices, conf_matrix, model

In [ ]:
mismatched_indices, conf_matrix, model_haha = bert_experiment(16, 5e-5, 2)

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were n


	 - Train loss: 0.1069

	 - Validation accuracy (epoch): 0.9740
              precision    recall  f1-score   support

         0.0       0.96      0.99      0.97      4159
         1.0       0.99      0.96      0.97      4087

    accuracy                           0.97      8246
   macro avg       0.97      0.97      0.97      8246
weighted avg       0.97      0.97      0.97      8246



Epoch: 100%|██████████| 2/2 [09:55<00:00, 297.84s/it]


	 - Train loss: 0.0399

	 - Validation accuracy (epoch): 0.9762
              precision    recall  f1-score   support

         0.0       0.97      0.98      0.98      4159
         1.0       0.98      0.97      0.98      4087

    accuracy                           0.98      8246
   macro avg       0.98      0.98      0.98      8246
weighted avg       0.98      0.98      0.98      8246

(array([  74,  120,  151,  230,  255,  286,  287,  344,  368,  390,  492,
        527,  594,  654,  656,  672,  677,  683,  720,  768,  772,  779,
        795,  884,  888,  908,  923,  967, 1036, 1052, 1086, 1184, 1193,
       1216, 1227, 1279, 1286, 1294, 1295, 1341, 1522, 1613, 1623, 1871,
       1946, 1948, 1952, 1967, 1974, 1996, 1999, 2040, 2041, 2061, 2122,
       2188, 2309, 2315, 2335, 2379, 2453, 2494, 2530, 2555, 2591, 2704,
       2734, 2836, 2841, 2849, 2911, 2913, 2923, 2930, 2952, 2985, 2987,
       3028, 3048, 3089, 3150, 3188, 3206, 3333, 3380, 3393, 3411, 3509,
       3529, 3558, 3656

In [ ]:
model = BertForSequenceClassification.from_pretrained(
    "DeepPavlov/rubert-base-cased",
    num_labels = 2,
    output_attentions = False,
    output_hidden_states = False
    )

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were n

In [ ]:
sample = 'Колобок долго думал и все-таки решил повеситься.'
encoding_dict = preprocessing(sample, tokenizer)
eval_output = model(encoding_dict['input_ids'], token_type_ids = None, attention_mask = encoding_dict['attention_mask'])
logits = np.argmax(eval_output.logits.detach().cpu().numpy(), axis = 1).flatten()
logits[0]

0

In [ ]:
import pickle

filename = 'haha-model.sav'
pickle.dump(model_haha, open(filename, 'wb'))